**Mount your drive in order to run locally with colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

**Move to the project folder path**

In [ ]:
cd "/content/drive/MyDrive/medical_image_processing_using_DP/project_204452072_203737739/"

**Set all relevance parameters**

* img_size - the size of the images and masks after resizing. Both networks (segmentation and classification) receives inputs image and mask of this size (fix- should not change).
* split_dataset_to_N - split the dataset to 3 parts (sub-folders) in order to handle with the memory constraints of Google Colab (fix- should not change).
* how_to_separate_dataset -  "random" for random approach or "patient" for subject approach (can change: "random" or "patient" but need to stay fix for both networks).
* num_classes - number of classes for the 3 sub-classification networks, each one of them classify between 2 tumors types (fix- should not change).  


In [25]:
img_size = (256,256)
split_dataset_to_N = 3
how_to_separate_dataset="patient" # can be random or patient
num_classes = 2

**Training part**:

*Train segmentation Unet NN*:

*save weights path:*

Insert the path where you want to save the weights for the segmentation network, for example:

In [20]:
segmentation_model_save_path ="saved_classifier_model/example_save_weights_file_unet_model_" + how_to_separate_dataset + "_separation.h5"

*Train segmentation model*:

- epochs_segmentation_model - number of epochs (can change).
- batch_size_segmentation_model - number of images per betch (can change).

In [ ]:
from tumor_classification_NN import train_Unet_segmentation_model
epochs_segmentation_model = 100
batch_size_segmentation_model = 32

train_Unet_segmentation_model(segmentation_model_save_path,how_to_separate_dataset,epochs_segmentation_model,batch_size_segmentation_model,img_size)

*Train clasification model (contain 3 sub models)*:

save weights path:

Insert the path where you want to save the weights for the segmentation network, for example:

In [21]:
classification_model_save_path = "saved_classifier_model/example_save_weights_file_clasification_model_"+how_to_separate_dataset+"_separation_"

*Train classification model*:

- epochs_segmentation_model - number of epochs (can change).
- batch_size_segmentation_model - number of images per betch (can change).
- with_mask - train the classification networks with (True) or without (False) the masks.
- itr - Because we have memory contrian in Google colab we had to split our train set into 3 (split_dataset_to_N = 3) sub sets.
itr/split_dataset_to_N - is the number of iteration we train each one of the 3  sub sets, in each iteration we train the sub set with 30 epochs (epochs_classification_model). total number of epochs on the entire train set is equal to (itr/split_dataset_to_N) x epochs_classification_model = 5x30 = 150

In [ ]:
from tumor_classification_NN import train_all_classification_sub_models
epochs_classification_model = 30
batch_size_classification_model = 32
with_mask = True
itr = split_dataset_to_N * 5 #(total 5*30=150 epochs)

train_all_classification_sub_models(epochs_classification_model, batch_size_classification_model, img_size,num_classes, with_mask, classification_model_save_path, split_dataset_to_N, itr, how_to_separate_dataset)

**Testing part- test entire Model - random approach:**

load model weights:

Insert the path where the all wheights are saved in your google drive.

for example, in our drive this folder sits at the following path:

In [14]:
segmentation_model_load_path="saved_classifier_model/unet_segmentation_model_random_separation.h5"
classification_model_load_path="saved_classifier_model/Model_with_pred_mask_random_separation_"

*Creating all the relevance networks and load weights for them* (segmentation and classification).
* use_pred_mask- 

  - True for using the masks created by the the segmentation network. 

  - False- for using the original masks (Groung Truth masks).

In [ ]:
from tumor_classification_NN import test_model,create_classification_model,unet_model
use_pred_mask = True
model_label_1_2 = create_classification_model(img_size, num_classes, with_mask)
model_label_1_3 = create_classification_model(img_size, num_classes, with_mask)
model_label_2_3 = create_classification_model(img_size, num_classes, with_mask)
model_label_1_2.load_weights(classification_model_load_path+"label_1_2.h5")
model_label_1_3.load_weights(classification_model_load_path+"label_1_3.h5")
model_label_2_3.load_weights(classification_model_load_path+"label_2_3.h5")
segmentation_model = unet_model(img_size + (1,), use_attention=True)
segmentation_model.load_weights(segmentation_model_load_path)

test_model(segmentation_model, model_label_1_2, model_label_1_3, model_label_2_3, img_size, with_mask,"random",use_pred_mask)

**Testing part- test entire Model - subject approach:**

*load model weights*:

Insert the path where the all wheights are saved in your google drive. 

for example, in our drive this folder sits at the following path:

In [16]:
segmentation_model_load_path="saved_classifier_model/unet_segmentation_model_patient_separation.h5"
classification_model_load_path="saved_classifier_model/Model_with_pred_mask_patient_separation_"

*Creating all the relavance networks and load weights for them* (segmentation and classification).

In [ ]:
from tumor_classification_NN import test_model,create_classification_model,unet_model
use_pred_mask = True
model_label_1_2 = create_classification_model(img_size, num_classes, with_mask)
model_label_1_3 = create_classification_model(img_size, num_classes, with_mask)
model_label_2_3 = create_classification_model(img_size, num_classes, with_mask)
model_label_1_2.load_weights(classification_model_load_path+"label_1_2.h5")
model_label_1_3.load_weights(classification_model_load_path+"label_1_3.h5")
model_label_2_3.load_weights(classification_model_load_path+"label_2_3.h5")
segmentation_model = unet_model(img_size + (1,), use_attention=True)
segmentation_model.load_weights(segmentation_model_load_path)

test_model(segmentation_model, model_label_1_2, model_label_1_3, model_label_2_3, img_size, with_mask,"patient",use_pred_mask)